In [2]:
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

tf.get_logger().setLevel(logging.ERROR)

In [3]:
data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1).sort_values('ID', ascending=True)
data.head()

,CroppedHexBugCoordinationX,CroppedHexBugCoordinationY,OriginalBoxCoordinationX1,OriginalBoxCoordinationX2,OriginalBoxCoordinationY1,OriginalBoxCoordinationY2,Path,ID
497,197,95,923,1223,91,391,cropped_bugs/training04/frame0.jpg,0
396,41,31,591,891,218,518,cropped_bugs/training05/frame0.jpg,0
201,90,154,140,440,374,674,cropped_bugs/training01/frame0.jpg,0
100,197,95,908,1208,581,881,cropped_bugs/training03/frame0.jpg,0
302,104,175,66,366,263,563,cropped_bugs/training06/frame0.jpg,0


In [4]:
# Load the images
images = []
y = []

for idx, row in data.iterrows():
    img_path = f'./{row.Path}'
    annotations = [int(row.CroppedHexBugCoordinationX), int(row.CroppedHexBugCoordinationY)]
    img = load_img(img_path)
    np_img = np.array(img)
    images.append(np_img)
    y.append(annotations)
    
X = np.array(images)
y = np.array(y)
print(X.shape)
print(y.shape)

(498, 300, 300, 3)
(498, 2)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [6]:
print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')
print(f'y train shape: {y_train.shape}')
print(f'y test shape: {y_test.shape}')

X train shape: (398, 300, 300, 3)
X test shape: (100, 300, 300, 3)
y train shape: (398, 2)
y test shape: (100, 2)


### Create CNN Model

In [24]:
# # del model

# model = Sequential()
# model.add(Conv2D(128, (3, 3), activation='relu', input_shape=X_train[0].shape))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(1028, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(2, activation='linear'))

In [26]:
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=X_train[0].shape)

for layer in resnet50.layers:
    layer.trainable = True

# Add a new classification layer for regression
x = resnet50.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(2, activation='linear')(x)

model = Model(inputs=resnet50.input, outputs=output_layer)

In [27]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

/usr/bin/zsh: /home/farzam/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)


In [28]:
checkpoint_filepath = './checkpoint_callback'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [29]:
model.compile(loss='mse', optimizer='adam')

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 75, 75, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 38, 38, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 19, 19, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                                                  
 conv5_block1_add (Add)         (None, 10, 10, 2048  0           ['conv5_block1_0_bn[0][0]',      
                                )                                 'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 10, 10, 2048  0           ['conv5_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 10, 10, 512)  1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 10, 10, 512)  2048       ['conv5_block2_1_conv[0][0]']    
 ization) 

In [32]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_checkpoint_callback])

In [14]:
%tensorboard --logdir logs/fit

In [17]:
def compute_error(y, y_pred):
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [18]:
import random

num_samples = 10

random_samples_indices = random.sample(range(0, len(X_test)), num_samples)

print(random_samples_indices)

for sample_index in random_samples_indices:
    sample_img = X_test[sample_index]
    
    center = y_test[sample_index]
    radius = 5
    color = (0, 255, 0)
    color_pred = (0, 0, 255)
    
    thickness = 5
    
    expanded_img = np.expand_dims(sample_img, axis=0)
    
    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
    
    error = round(compute_error(center, predicted), 3)

    print(f'y_predicted: {predicted}, y: {center}, Error: {error}')
    
    img_lv1 = cv2.circle(sample_img, center, radius, color, thickness)
    img_lv2 = cv2.circle(img_lv1, predicted, radius, color_pred, thickness)

    cv2.imshow(f'Sample No. {sample_index}, error: {error}', img_lv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[24, 32, 95, 18, 1, 14, 69, 52, 50, 66]


2023-04-25 09:41:01.088240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-25 09:41:01.524480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 2s 2s/step
y_predicted: [269, 25], y: [272  26], Error: 3.162
1/1 [==============================] - 0s 14ms/step
y_predicted: [38, 271], y: [ 37 264], Error: 7.071
1/1 [==============================] - 0s 13ms/step
y_predicted: [44, 34], y: [46 35], Error: 2.236
1/1 [==============================] - 0s 14ms/step
y_predicted: [49, 35], y: [45 23], Error: 12.649
1/1 [==============================] - 0s 14ms/step
y_predicted: [238, 40], y: [215  20], Error: 30.48
1/1 [==============================] - 0s 13ms/step
y_predicted: [205, 190], y: [198 185], Error: 8.602
1/1 [==============================] - 0s 13ms/step
y_predicted: [145, 266], y: [134 268], Error: 11.18
1/1 [==============================] - 0s 14ms/step
y_predicted: [97, 307], y: [ 91 298], Error: 10.817
1/1 [==============================] - 0s 14ms/step
y_predicted: [31, 166], y: [ 33 165], Error: 2.236
1/1 [==============================] - 0s 15ms/step
y_predicted: [49, 31], y: